In [1]:
### Web Scraping Stack Overflow - Part II - Multiple pages

### Base code modified and expanded from
### https://medium.com/better-programming/how-to-scrape-multiple-pages-of-a-website-using-a-python-web-scraper-4e2c641cff8

In [2]:
# Import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup as bs
import requests

%matplotlib inline

In [3]:
#user_span_single = user_spans[0]

def user_find(user_span_single):
    
    trial = user_span_single.select('a')
    
    if len(trial) >= 1:
    
        intermediate = trial[0].text
        
        if '\r\n' in intermediate:
        
            return None
        
        return intermediate
    
    if 'anon' in user_span_single.text:
    
        return 'anonymous'
    
    else:
        check = user_span_single.get_text(strip=True)
        
        if not check:
        
            return 'no_user_info'
    
        return check

In [4]:
#time_spans = body.select("span.relativetime")
#time_stamp = [i["title"] for i in time_spans]

def time_find(users, time_stamp):
 
    polished_times = []
    times_iter = iter(time_stamp)

    for user_id in users:
        if user_id == 'community wiki' or user_id == 'anonymous':
            polished_times.append('None') 
        
        else:
            polished_times.append(next(times_iter))
            
    
    return polished_times

In [ ]:
for i in range(672,2500):

    questions = []
    summaries = []
    tags = []
    times = []
    users = []
    no_of_votes = []
    no_of_answers = []
    failed = []
    
    pages = np.arange((i * 100 + 1) - 100, i * 100 + 1, 1)

    
    try:
    
        for page in pages:

            #def get_top_questions(page):
            # WARNING: Only enter one of these 3 values [15, 30, 50].
            # Since, stackoverflow, doesn't display any other size questions list
            #url = url + "?sort=votes&pagesize={}".format(question_count)

            # Using requests module for downloading webpage content
            page = requests.get("https://stackoverflow.com/questions?tab=active&page=" + str(page))

            # Parsing html data using BeautifulSoup
            soup = bs(page.content, 'html.parser')
            body = soup.find('body')

            # Extracting Top Questions
            question_links = body.select("h3 a.question-hyperlink")
            #error_checking(question_links, question_count)                     # Error Checking
            questions.extend([i.text for i in question_links])                      # questions list

            # Extracting Summary
            summary_divs = body.select("div.excerpt")
            #error_checking(summary_divs, question_count)                       # Error Checking
            summaries.extend([i.text.strip() for i in summary_divs])                 # summaries list

            # Extracting Tags
            tags_divs = body.select("div.summary > div:nth-of-type(2)")

            #error_checking(tags_divs, question_count)                          # Error Checking
            a_tags_list = [i.select('a') for i in tags_divs]                   # tag links

            tagged_tags = []

            for a_group in a_tags_list:
                    tagged_tags.append([a.text for a in a_group])                         # tags list

            tags.extend(tagged_tags)

            # Extracting User Info
            user_spans = body.select("div.user-details")
            user_found = [user_find(user_span_single) for user_span_single in user_spans]
            users_list = [x for x in user_found if x]
            users.extend([x for x in user_found if x])
            #print(len(users))

            # Extracting Question Time Stamps
            time_spans = body.select("span.relativetime")
            time_stamp = [i["title"] for i in time_spans]
            times.extend(time_find(users_list, time_stamp))

            #print(len(time_spans))                                             # time list

            # Extracting Number of votes
            vote_spans = body.select("span.vote-count-post strong")
            #error_checking(vote_spans, question_count)                         # Error Checking
            no_of_votes.extend([int(i.text) for i in vote_spans])                    # votes list

            # Extracting Number of answers
            answer_divs = body.select("div.status strong")
            #error_checking(answer_divs, question_count)                        # Error Checking
            no_of_answers.extend([int(i.text) for i in answer_divs])                 # answers list

        # Putting all of them together
        test_df = pd.DataFrame({'question': questions, 
                                'summary': summaries, 
                                'tags': tags,
                                'time_stamp': times,
                                'user_id': users,
                                'no_of_votes': no_of_votes,
                                'no_of_answers': no_of_answers})

        test_df.to_csv('scrape/test_scrape_{}.csv'.format(str(i).zfill(4)))

        print(i) 
        
    except:
        failed.append(i)

        failed_df = pd.DataFrame({'location': failed})
        failed_df.to_csv('failed_df.csv')

672
673
674
675
676
677
678
679
680
681
682
683
684
685
686
687
688
689
690
691
692
693
694
695
696
697
698
699
700
701
702
703
704
705
706
707
708
709
710
711
712
713
714
715
716
717
718
719
720
721
722
723
724
725
726
727
728
729
730
731
732
733
734
735
736
737
738
739
740
741
742
743
744
745
746
747
748
749
750
751
752
753
754
755
756
757
758
759
760
761
762
763
764
765
766
767
768
769
770
771
772
773
774
775
776
777
778
779
780
781
782
783
784
785
786
787
788
789
790
791
792
793
794
795
796
797
798
799
800
801
802
803
804
805
806
807
808
809
810
811
812
813
814
815
816
817
818
819
820
821
822
823
824
825
826
827
828
829
830
831
832
833
834
835
836
837
838
839
840
841
842
843
844
845
846
847
848
849
850
851
852
853
854
855
856
857
858
859
860
861
862
863
864
865
866
867
868
869
870
871
872
873
874
875
876
877
878
879
880
881
882
883
884
885
886
887
888
889
890
891
892
893
894
895
896
897
898
899
900
901
902
903
904
905
906
907
908
909
910
911
912
913
914
915
916
917
918
919
920
921


In [ ]:
# Fail Analysis

In [ ]:
len(times)

In [ ]:
len(users)

In [ ]:
len(no_of_answers)